In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
df = pd.read_csv('nba_game_df.csv')
df = df.sort_values(by=['date']).reset_index(drop=True)
pd.set_option('display.max_columns', None)
df.head(6)

,Team_A,Team_B,matchup_id,date,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_USG%_advanced_max,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_ORB%_advanced_max,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_USG%_advanced_max,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win,Team_A_home,Team_A_win_next,playoff_game
0,12,13,LAC_vs_LAL,2009-10-27,14.0,1.0,0.200,5.0,15.0,1.0,0.172,2.0,3.0,38.8,69.2,8.0,27.0,3.5,5.9,1.0,4.0,34.4,67.9,14.0,36.0,110.0,101.6,1.000,0.448,17.0,87.0,8.0,39.0,1.000,0.688,6.0,16.0,5.0,11.0,0.429,0.184,15.1,240.0,19.2,33.3,6.0,15.0,133.0,94.4,4.0,27.0,21.0,92.0,22,27,26,17,3.6,10.3,2.0,10.0,50.0,17.5,4.0,20.0,22.0,52.0,16.0,51.0,1.500,0.489,27.4,1.500,0.466,14.0,0.400,0.235,5.0,17.0,0.500,0.200,2.0,4.0,21.5,48.6,5.0,17.0,3.3,5.6,2.0,4.0,26.6,10.0,30.0,102.0,94.4,0.600,0.412,26.0,85.0,11.0,35.0,0.917,0.676,12.0,37.0,11.0,25.0,0.700,0.435,26.8,18.5,6.0,17.0,136.0,101.6,3.0,15.0,33.0,99.0,32,27,17,23,14.6,13.3,4.0,13.0,33.3,13.6,4.0,16.0,16.7,13.0,47.0,0.670,0.489,35.5,0.600,0.435,0,0,1.0,0
1,10,24,HOU_vs_POR,2009-10-27,8.0,1.0,0.278,6.0,18.0,0.5,0.222,2.0,5.0,58.7,60

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [4]:
df = df.drop(columns=['date', 'matchup_id'])
train_size = int(0.7 * len(df))  
val_size = int(0.15 * len(df))   
test_size = len(df) - train_size - val_size  

y = df['Team_A_win_next']
df = df.drop(columns=['Team_A_win_next'])
x_train = df.iloc[:train_size]

x_val = df.iloc[train_size:train_size+val_size]
x_test = df.iloc[train_size+val_size:]
y_train = y.iloc[:train_size]
y_train = y_train.to_numpy().reshape(-1,1)
y_val = y.iloc[train_size:train_size+val_size]
y_val = y_val.to_numpy().reshape(-1,1)
y_test = y.iloc[train_size+val_size:]
y_test = y_test.to_numpy().reshape(-1,1)
x_train.shape

(13047, 141)

In [5]:
all_columns = df.columns.tolist()
binary_cols = [col for col in x_train.columns if x_train[col].nunique() == 2]
exclude_cols = set(binary_cols + ['Team_A', 'Team_B'])
normalize_cols = [col for col in all_columns if col not in exclude_cols]
normalize_indices = [all_columns.index(col) for col in normalize_cols]

mean = x_train[normalize_cols].mean().values
std = x_train[normalize_cols].std().values
x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
x_test = x_test.to_numpy()

x_train[:, normalize_indices] = (x_train[:, normalize_indices] - mean) / std
x_val[:, normalize_indices] = (x_val[:, normalize_indices] - mean) / std
x_test[:, normalize_indices] = (x_test[:, normalize_indices] - mean) / std

In [6]:
num_features = x_train.shape[1]
w = np.random.normal(0, 0.01, size=(num_features, 1))
b = 0
#z = x_train @ w + b
#y_pred = sigmoid(z)

In [7]:
#print(y_pred[:5])         # Should show values between 0 and 1
#print(y_pred.shape)       # Should be (m_train, 1)

In [8]:
def compute_cost(x, y, w, b):
    m = y.shape[0]
    z = x @ w + b
    y_pred = sigmoid(z)
    loss = - (1/m) * (y.T @ np.log(y_pred) + (1 - y).T @ np.log(1 - y_pred))
    return loss.item()

In [9]:
def compute_gradients(x, y, w, b): 
    m = y.shape[0]
    z = x @ w + b
    y_pred = sigmoid(z)
    dj_dw = (1/m) * x.T @ (y_pred - y) 
    dj_db = (1/m) * np.sum(y_pred - y)
    return dj_dw, dj_db

In [10]:
learning_rate = 0.01
num_epochs = 100
train_cost_history = []
val_cost_history = []

for epoch in range(num_epochs):
    dj_dw, dj_db = compute_gradients(x_train, y_train, w, b)
    
    #print(f"Epoch {epoch+1}")
    #print(f"  Mean dj_dw: {np.mean(dj_dw):.6f}, dj_db: {dj_db:.6f}")
    #print(f"  Mean weight before update: {np.mean(w):.6f}")

    w = w - (learning_rate * dj_dw)
    b = b - (learning_rate * dj_db)
    
    #print(f"  Mean weight after update: {np.mean(w):.6f}")
    #print(f"  Training Cost: {train_cost:.6f}, Validation Cost: {val_cost:.6f}")
    
    train_cost = compute_cost(x_train, y_train, w, b)
    train_cost_history.append(train_cost) 
    val_cost = compute_cost(x_val, y_val, w, b)
    val_cost_history.append(val_cost) 
    print(f"Epoch {epoch+1}/{num_epochs}, Training Cost: {train_cost:.4f}, Validation Cost: {val_cost:.4f}")


Epoch 1/100, Training Cost: 0.6988, Validation Cost: 0.7008
Epoch 2/100, Training Cost: 0.6956, Validation Cost: 0.6959
Epoch 3/100, Training Cost: 0.6945, Validation Cost: 0.6956
Epoch 4/100, Training Cost: 0.6937, Validation Cost: 0.6947
Epoch 5/100, Training Cost: 0.6929, Validation Cost: 0.6942
Epoch 6/100, Training Cost: 0.6922, Validation Cost: 0.6936
Epoch 7/100, Training Cost: 0.6916, Validation Cost: 0.6931
Epoch 8/100, Training Cost: 0.6910, Validation Cost: 0.6927
Epoch 9/100, Training Cost: 0.6904, Validation Cost: 0.6922
Epoch 10/100, Training Cost: 0.6898, Validation Cost: 0.6918
Epoch 11/100, Training Cost: 0.6892, Validation Cost: 0.6914
Epoch 12/100, Training Cost: 0.6887, Validation Cost: 0.6910
Epoch 13/100, Training Cost: 0.6882, Validation Cost: 0.6907
Epoch 14/100, Training Cost: 0.6878, Validation Cost: 0.6903
Epoch 15/100, Training Cost: 0.6873, Validation Cost: 0.6900
Epoch 16/100, Training Cost: 0.6869, Validation Cost: 0.6897
Epoch 17/100, Training Cost: 0.68

In [11]:
z_val = x_val @ w + b
y_val_probs = sigmoid(z_val)
y_val_pred = (y_val_probs >= 0.5).astype(int)
accuracy = np.mean(y_val_pred == y_val)
print(f"Validation Accuracy: {accuracy * 100:.3f}%")

Validation Accuracy: 55.456%


In [12]:
conf_matrix = confusion_matrix(y_val, y_val_pred)
conf_matrix_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_qda_display.plot()

NameError: name 'cm_qda_display' is not defined

In [ ]:
tn, fp, fn, tp = conf_matrix.ravel()

tpr = tp / (tp + fn)  
fpr = fp / (fp + tn)  
tnr = tn / (tn + fp)  
fnr = fn / (fn + tp)  
print(f"TPR: {tpr:.4f}")
print(f"FPR: {fpr:.3f}")
print(f"TNR: {tnr:.3f}")
print(f"FNR: {fnr:.3f}")
probs = sigmoid(x_val @ w + b)
auc = roc_auc_score(y_val, probs)
print("AUC:", auc)

In [ ]:
#p = 0.45
#TPR: 0.7830
#FPR: 0.667
#TNR: 0.333
#FNR: 0.217
#AUC: 0.5759889392426454
#Validation Accuracy: 55.993%

#p = 0.5 
#TPR: 0.5702
#FPR: 0.466
#TNR: 0.534
#FNR: 0.430
#AUC: 0.5781007245820211
#Validation Accuracy: 55.242%